### Cora dataset

In [91]:
# cora dataset = 1 graph of citation network: node classification dataset
import dgl.data
dataset=dgl.data.CoraGraphDataset()
g=dataset[0]

# train_mask, val_mask, test_mask = boolean indices for train, val, test
train_mask=g.ndata['train_mask']
val_mask=g.ndata['val_mask']
test_mask=g.ndata['test_mask']

print(f"num_train_nodes: {sum(train_mask)} | num_val_nodes: {sum(val_mask)} | num_test_nodes: {sum(test_mask)}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
num_train_nodes: 140 | num_val_nodes: 500 | num_test_nodes: 1000


In [92]:
# hyperparameters (used on models below)
in_dim=g.ndata['feat'].shape[-1]
hidden_dim=16
out_dim=dataset.num_classes
num_heads=2

### 1. Graph Attention Network
GCN combines local graph structure and node-level features. 
$$h_i^{(l+1)}=W\sum_{j\in N(i)}\dfrac{1}{\sqrt{\deg(i)\deg(j)}}h_j^{(l)}+b$$
This approach yields good performance on node classification tasks. However, GCN only considers neighbor information and ignores the node itself. Such a structure dependent approach can hurt its generality (graph calssification).

GraphSage remedies this issue by concatenating the node and the neighbor information.
$$h_i^{(l+1)}= W\text{concat}(h_i^{(l)},h_{N(i)}^{(l+1)})+b \ \text{with} \ h_{N(i)}^{(l+1)}=\text{Mean}\{h_j^{(l)}: j\in N(i)\}$$

GAT uses weighting neighbor features with feature dependent and structure-free normalization, in the style of attention, i.e. the attention coefficients are learnable!

\begin{align}
h_i^{(l+1)} &=W\sum_{j\in N(i)}\alpha_{ij}^{(l)}h_j^{(l)} \ \text{with} \\
\alpha_{ij}^{(l)}&=\text{softmax}_j(e_{ik}^{(l)}: k \in N(i)) \\
e_{ij}^{(l)}&= \text{LeakyReLU}\left(a^{(l)T}\cdot\text{concat}(Wh_i^{(l)} , Wh_j^{(l)})\right)
\end{align}
Note: $e_{ij}$ = un-normalized attention of edge $\{i,j\}$, $\alpha_{ij}$ = normalized attention coefficients.

### 2. GAT layer from scratch

#### (a) Single-head GAT Layer

In [93]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F 

class GAT_layer(nn.Module):
    def __init__(self,in_dim,out_dim):
        super().__init__()

        # matrix W
        self.linear_proj=nn.Linear(in_dim,out_dim,bias=False)

        # attention params a
        self.attn_param=nn.Linear(2*out_dim,1,bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        # initialize learnable parameters by xavier_normal with "relu gain"
        nn.init.xavier_normal_(self.linear_proj.weight,gain=nn.init.calculate_gain('relu'))
        nn.init.xavier_normal_(self.attn_param.weight,gain=nn.init.calculate_gain('relu'))
    
    # user-defined function for equation (3)
    #              return {'e': e} stored in edges.data: via g.apply_edges
    def edge_attention(self,edges): 
        # concatenate src and dst node features
        concat=torch.cat([edges.src['W_h'], edges.dst['W_h']],dim=1) # [E,2d] 
        e=self.attn_param(concat)                                    # [E,out_dim]
        return {'e': F.leaky_relu(e)}                                # [E,out_dim]

    # message_func(self, edges): send info through edges
    #                            return all information needed to update node features 
    #                            {'W_h':W_h, 'e': e} that is stored in nodes.mailbox
    def message_func(self,edges):
        return {'W_h': edges.src['W_h'], 'e': edges.data['e']}  # nodes.mailbox['W_h']=[E,in_dim]
        

    # reduce_func(self,nodes): update node features in equation (1)
    #                          return {'h':h} that is stored in 
    def reduce_func(self,nodes):
        # attention coefficients
        alpha=F.softmax(nodes.mailbox['e'],dim=1)               # [E,out_dim]

        # take weighted sum of the neighbors
        h_N=torch.sum(alpha * nodes.mailbox['W_h'],dim=1)       # [E,out_dim]*[E,out_dim]=[E,out_dim]
        
        return {'h_N': h_N} # new node features
        

    def forward(self,g,h):
        
        with g.local_scope():

            g.ndata['h']=h                                      # [N,in_dim]
            
            W_h=self.linear_proj(h)
            g.ndata['W_h']=W_h                                   # [N,in_dim]
            
            # assign attention e_ij to every edges
            g.apply_edges(self.edge_attention) 

            # send info through edges and update node features
            g.update_all(self.message_func,self.reduce_func)

            return g.ndata['h_N']
 




In [94]:
# sanity check
lay=GAT_layer(in_dim,out_dim)
out=lay(g,g.ndata['feat'])
print(out.shape)

torch.Size([2708, 7])


#### (b) Multi-head GAT Layer

In [95]:
class MultiHeadGAT_Layer(nn.Module):
    def __init__(self,in_dim,out_dim,num_heads):
        super().__init__()
        self.num_heads=num_heads
        self.heads=nn.ModuleList([GAT_layer(in_dim,out_dim) for _ in range(num_heads)])

    def forward(self,g,h):
        # concatenate individual outputs
        out=torch.cat([self.heads[i](g,h) for i in range(self.num_heads)],dim=1)
        return out
    

In [96]:
# sanity check
layer_multihead=MultiHeadGAT_Layer(in_dim,out_dim,num_heads)
output=layer_multihead(g,g.ndata['feat'])
print(output.shape)

torch.Size([2708, 14])


#### (c) Model
input -> MultiHeadGAT_Layer1 -> elu -> MultiHeadGAT_Layer2 -> out

In [97]:
class GAT_Net(nn.Module):
    def __init__(self,in_dim,hidden_dim,out_dim,num_heads):
        super().__init__()
        self.gat1=MultiHeadGAT_Layer(in_dim,hidden_dim,num_heads)    # output dim=hidden_dim*num_heads
        self.gat2=MultiHeadGAT_Layer(hidden_dim*num_heads,out_dim,1) # output dim = out_dim*1=out_dim

    def forward(self,g,h):
        h=self.gat1(g,h)
        h=F.elu(h)
        h=self.gat2(g,h)
        return h
    

### 3. Train

In [98]:
# train loop

def train(model, graph, loss_fn,optimizer):
    model.train()
    features=graph.ndata['feat']
    labels=graph.ndata['label']
    
    # forward and backward on train_mask    
    optimizer.zero_grad()

    # prediction on the whole graph
    logits=model(graph, features) 
    
    # only consider train_mask
    loss=loss_fn(logits[train_mask],labels[train_mask])
    loss.backward()
    optimizer.step()

    # compute accuracy
    preds=logits.argmax(dim=-1)
    acc=(preds[train_mask]==labels[train_mask]).float().mean()
    return loss, acc

@torch.no_grad()
def evaluate(model,graph, loss_fn,optimizer):
    model.eval()
    features=graph.ndata['feat']
    labels=graph.ndata['label']

    # forward
    logits=model(graph,features)
    loss=loss_fn(logits[val_mask],labels[val_mask])

    # compute accuracy
    preds=logits.argmax(dim=-1)
    acc=(preds[val_mask]==labels[val_mask]).float().mean()
    return loss, acc


In [99]:
torch.manual_seed(1442)

num_epochs=20

# model and optimizer

model=GAT_Net(in_dim,hidden_dim,out_dim,num_heads)
print(f"{sum(p.numel() for p in model.parameters())/1e6} million parameters")

loss_fn=F.cross_entropy
optimizer=torch.optim.AdamW(model.parameters(),lr=0.1)


# train and test
for epoch in range(num_epochs):
    train_loss, train_acc=train(model,g,loss_fn,optimizer)
    val_loss, val_acc=evaluate(model,g,loss_fn,optimizer)
    if epoch%5==0 or epoch == num_epochs-1:
        print(f"Epoch : {epoch} | train_loss : {train_loss:.4f} | train_acc : {train_acc*100:.2f}% | "
            f" val_loss : {val_loss:.4f} | val_acc : {val_acc*100:.2f}%")

0.046158 million parameters
Epoch : 0 | train_loss : 1.9467 | train_acc : 12.86% |  val_loss : 1.8070 | val_acc : 73.40%
Epoch : 5 | train_loss : 0.3415 | train_acc : 97.14% |  val_loss : 0.8541 | val_acc : 75.80%
Epoch : 10 | train_loss : 0.0145 | train_acc : 100.00% |  val_loss : 1.2353 | val_acc : 75.60%
Epoch : 15 | train_loss : 0.0016 | train_acc : 100.00% |  val_loss : 1.5936 | val_acc : 76.00%
Epoch : 19 | train_loss : 0.0034 | train_acc : 100.00% |  val_loss : 1.7326 | val_acc : 76.40%
